# Imports

In [ ]:
#the following ten imports have to be installed
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorboard.plugins.hparams import api as hp
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import seaborn as sns

import os 
import shutil
import pandas.util
import math

# Hyperparameters

In [ ]:
IMG_SIZE = (105,75)    # width and height of all images (resize, if required)
BATCH_SIZE = 32  # for training and prediction
EPOCHS = 20 #number of epochs for training
NUM_RUNS = 1 #how many times should the training be repeated with the same hyperparameters?

HP_DROPOUT_CONV = hp.HParam('dropout_conv', hp.Discrete([.3]))
HP_DROPOUT_DENSE = hp.HParam('dropout_dense', hp.Discrete([0.3]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
METRIC_ACCURACY = 'accuracy'
LOG_DIR = 'logs/'

TEST_PKL = "files/classification_test.pkl"
TRAIN_PKL = "files/classification_train.pkl"
VAL_PKL = "files/classification_val.pkl"

## Check if all files and folder exist 

In [ ]:
# If the LOG_DIR path does not exist, create it. 
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
# However, if the path exists and has been used before, the content has to be deleted and the directory created again!
if os.path.exists(LOG_DIR):
    shutil.rmtree(LOG_DIR)
    os.mkdir(LOG_DIR)

#check if the TEST_PKL, TRAIN_PKL, VAL_PKL files exist. If not: raise an error.
if not os.path.exists(TEST_PKL):
    raise FileNotFoundError('The model preprocess (classification_model_preprocess.ipynb) has to be conducted first or the name of the TEST_PKL has to be adjusted.')
if not os.path.exists(TRAIN_PKL):
    raise FileNotFoundError('The model preprocess (classification_model_preprocess.ipynb) has to be conducted first or the name of the IMAGE_FOLDER has to be adjusted.')
if not os.path.exists(VAL_PKL):
    raise FileNotFoundError('The model preprocess (classification_model_preprocess.ipynb) has to be conducted first or the name of the IMAGE_FOLDER has to be adjusted.')

# Get test, train, validation datasets

In [ ]:
test_df = pd.read_pickle(TEST_PKL)
train_df = pd.read_pickle(TRAIN_PKL)
val_df = pd.read_pickle(VAL_PKL)

# Map a filename to an image tensor and one-hot encode label

In [ ]:
def path_to_array(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_png(img, channels = 3)
    # now img is 3 dim array of numbers in {0,..., 255}
    img = tf.cast(img, dtype = tf.float32) / 255. # scale to floating point number in [0,1] 
 
    # one-hot encode the label, e.g. 3 becomes [0,0,0,1,0,0,0,0,0,0,0,0]
    label = tf.one_hot(label, depth = 6) #6 classes
    return img, label

# Make a tf dataset of images from a pd data frame of file paths

In [ ]:
def make_dataset(df):
    # first, make dataset with just the relevant: path and class
    ds_path = tf.data.Dataset.from_tensor_slices((df['image_path'], df['class']))

    # convert to data set with actual images
    ds = ds_path.map(path_to_array)
    ds = ds.batch(BATCH_SIZE)
    return ds

test_ds  = make_dataset(test_df)
val_ds   = make_dataset(val_df)
train_ds = make_dataset(train_df)
train_ds = train_ds.repeat().prefetch(tf.data.experimental.AUTOTUNE) # infinitely repeat

# Determine the architecture of the model

In [ ]:
def create_model(hparams):
    
    kernel_size = (3, 3)
    pool_size   = (2, 2)
    first_filters  = 32
    second_filters = 64
    third_filters  = 128
    dropout_conv  = hparams[HP_DROPOUT_CONV]
    dropout_dense = hparams[HP_DROPOUT_DENSE]

    model = tf.keras.models.Sequential() # sequential stack of layers

    model.add( BatchNormalization(input_shape = (IMG_SIZE[1],IMG_SIZE[0], 3)))
    model.add( Conv2D (first_filters, kernel_size, activation = 'relu')) # convolutional layer + activation layer
    model.add( Conv2D (first_filters, kernel_size, activation = 'relu')) 
    model.add( Conv2D (first_filters, kernel_size, activation = 'relu')) 
    model.add( MaxPooling2D (pool_size = pool_size)) #Pooling layer
    model.add( Dropout (dropout_conv)) #Dropout layer

    model.add( Conv2D (second_filters, kernel_size, activation ='relu')) 
    model.add( Conv2D (second_filters, kernel_size, activation ='relu')) 
    model.add( Conv2D (second_filters, kernel_size, activation ='relu'))
    model.add( MaxPooling2D (pool_size = pool_size))
    model.add( Dropout (dropout_conv))

    model.add( Conv2D (third_filters, kernel_size, activation ='relu'))
    model.add( Conv2D (third_filters, kernel_size, activation ='relu'))
    model.add( Conv2D (third_filters, kernel_size, activation ='relu'))
    model.add( MaxPooling2D (pool_size = pool_size))
    model.add( Dropout (dropout_conv))

    model.add( Flatten())
    model.add( Dense (256, activation = "relu", kernel_regularizer = tf.keras.regularizers.l2(0.001))) #dense layer + activation layer
    model.add( Dropout (dropout_dense)) #Dropout layer
    model.add( Dense(6, activation = 'softmax') ) # activation layer

    return model

# Train the model

In [ ]:
num_train = len(train_df)
session_num = 0

for session_num in range(NUM_RUNS):
    for dr_conv in HP_DROPOUT_CONV.domain.values:
        for dr_dense in HP_DROPOUT_DENSE.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:

                hparams = {
                    HP_DROPOUT_CONV: dr_conv,
                    HP_DROPOUT_DENSE: dr_dense,
                    HP_OPTIMIZER: optimizer,
                }
                run_name = f"run-{session_num}"
                run_dir = f"{LOG_DIR}{run_name}"
                print(f'--- Starting trial: {run_name}')
                print({h.name: hparams[h] for h in hparams})


                with tf.summary.create_file_writer(run_dir).as_default():
                    hp.hparams(hparams)  # record the values used in this trial
                    model = create_model(hparams)
                    # Function to decrease learning rate by 'factor'
                    # when there has been no significant improvement in the last 'patience' epochs.
                    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', factor = 0.75, patience = 4, verbose = 1)
                    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_dir, histogram_freq=1, update_freq='batch')

                    # define the loss, optimization algorithm and prepare the model for gradient computation 
                    model.compile(optimizer = hparams[HP_OPTIMIZER],
                                  loss = 'categorical_crossentropy', metrics = [METRIC_ACCURACY]) 

                    # Callbacks: What should be done during training?
                    modelfname = f"model_checkpoints/classification_r{session_num}.h5"
                    # Function to store model to file, if validation loss has a new record
                    # Check always after having seen at least another save_freq examples.
                    checkpoint = tf.keras.callbacks.ModelCheckpoint(
                        modelfname, monitor = 'val_loss', mode = 'min', 
                        save_best_only = True, verbose = 1)
                    
                    #evaluate the model after each epoch
                    test_on_epoch_end = tf.keras.callbacks.LambdaCallback(
                        on_epoch_end=lambda epoch,logs: model.evaluate(test_ds, verbose = 1)
                    )
                    
                    history = model.fit_generator(
                        train_ds, epochs = EPOCHS, 
                        steps_per_epoch = num_train / BATCH_SIZE, #would use each example once on average
                        validation_data = val_ds, verbose = 1,
                        callbacks = [checkpoint,tensorboard_callback,test_on_epoch_end,]
                    )


# Evaluation

## Set and get parameters

In [ ]:
hparams = {HP_DROPOUT_CONV:0.3, HP_DROPOUT_DENSE:0.3}
hp.hparams(hparams)
model = create_model(hparams)
model.load_weights("model_checkpoints/classification_best.h5") #this is the h5 file of my best run

## Make a prediction on the test dataset

In [ ]:
prediction = model.predict(test_ds)
yhat = prediction.argmax(axis = 1)
if 'pred' not in test_df:
    test_df.insert(3, 'pred',  prediction.argmax(axis = 1))
if 'confidence' not in test_df:
    test_df.insert(4, 'confidence',  prediction.max(axis = 1))

## Create a confusion matrix

In [ ]:
# a confusion matrix with numbers as entries
con_mat = tf.math.confusion_matrix(labels=test_df['class'], predictions=test_df['pred']).numpy()
classnames = train_df['classname'].unique() 
K = classnames.size  # 6
name2class = dict(zip(classnames, range(K))) 

#create a dataframe as con_mat displays only a numpy array, meaning without column and row description
con_mat_df = pd.DataFrame(con_mat,
                     index = classnames, 
                     columns = classnames)

#normalized confusion matrix: only entries between 0 and 1
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
con_mat_norm_df = pd.DataFrame(con_mat_norm,
                     index = classnames, 
                     columns = classnames)
print(con_mat_norm_df)

## Function to print the confusion matrix as a heatmap

In [ ]:
from pylab import text

def print_confusion_matrix(confusion_matrix, class_names, figsize = (13,9), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt=".2f", vmin=0, vmax=1, center = 0.49,cmap=sns.cubehelix_palette(dark=0.4, light=1, as_cmap=True,rot=-.2,start=0))
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    heatmap.set_ylim(0,6)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    heatmap.text(7,4.5,f"Number of images per label:\n alp: {1200}\n tri: {420}\n zei: {660}\n iss: {1020}\n com: {420}\n oel: {360} ")
    return fig

### Print the confusion matrix

In [ ]:
fig = print_confusion_matrix(con_mat_norm_df, classnames) #print the normalized confusion matrix, otherwise change con_mat_norm_df to con_mat_df
#fig.savefig("heatmap.png")

## Calculate the classification report and print it

In [ ]:
class_report = classification_report(test_df['class'], test_df['pred'], target_names = classnames)
print(class_report)

## Determine false and correct classified images

In [ ]:
false_classified_df = test_df[(test_df['class'] != test_df['pred'])]
correct_classified_df = test_df[(test_df['class'] == test_df['pred'])]
numfalse = false_classified_df.shape[0]
print(f"number of false classified = {numfalse}")
numcorrect =correct_classified_df.shape[0]
print(f"number of correct classified = {numcorrect}")

## Plot some false classified images

In [ ]:
num_show = min(numfalse,50) # show at most 50 false examples
ncols = 4
nrows = math.ceil(num_show / ncols) # round up
nrows = min(nrows, 15) # at most 15 rows
f, ax = plt.subplots(nrows, ncols, figsize = (3 * ncols, 3 * nrows))
for k in range(num_show):
    i = math.floor(k / ncols) # row
    j = k % ncols # column
    record = false_classified_df.iloc[k]
    path = record['image_path']
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    ax[i, j].imshow(img)
    ax[i, j].set_title(classnames[record['class']] + " predicted as\n"+ 
                       str(classnames[record['pred']]) + " with conf. "
                       + str(np.round(record['confidence'], 3)),fontsize=12)

plt.tight_layout()